# HW 2 

### Problem 1 (3 points)

The ElGamal cryptosystem is based on the complexity of calculating the discrete logarithm for large primes 𝑝. In this task we will ask you to implement your own ElGamal encryption scheme on Python.

  1. Implement function for generating keys. The function must generate a random prime number (use Fermat primality test). (1 point)
  
*Note. You can assume that all the numbers are small, for example, less than $2^{32}$.*

In [45]:
def fast_pow(x, y):
    if y == 0:
        return 1
    val = fast_pow(x, int(y / 2))
    if y % 2 == 0:
        return val * val
    else:
        return x * val * val

def inv(a, q, x = 1):
    return fast_pow(a, q - 1 - x) % q


# i = 16
# print(inv(i, 1, 17))
# print((inv(i, 1, 17) * i) %17)


In [8]:
import random
def gcd(a, b):
    if a < b:
        return gcd(b, a)
    elif a % b == 0:
        return b;
    else:
        return gcd(b, a % b)

# Iterative Function to calculate 
# (a^n)%p in O(logy) 
def mod_power(a, n, p):
      
    # Initialize result 
    res = 1 
      
    # Update 'a' if 'a' >= p 
    a = a % p  
      
    while n > 0:
          
        # If n is odd, multiply 
        # 'a' with result 
        if n % 2:
            res = (res * a) % p
            n = n - 1
        else:
            a = (a ** 2) % p
              
            # n must be even now 
            n = n // 2
              
    return res % p

def prime_test(n, k=100):
    if n == 1:
        return False
    for i in range(k):
        if mod_power(random.randint(2, n - 2), n - 1, n) != 1:
            return False
    return True

def gen_prime(start = 7, stop = 23):
    n = random.randint(2, 11)
    while not prime_test(n):
        n = random.randint(2, 11)
    return n

gen_prime()

5

2. Implement functions that realize the encryption and decryption in ElGamal protocol. (1 point)

3. Encrypte message $M=4$, using open key $(y=8,g=6,p=13)$, private key $(x=3,g=6,p=13)$ and random parameter $K=11$ (1 point)

### Problem 2 (4 points)

There are analogues of ElGamal cryptosystems, in which instead of the discrete logarithm problem in multiplicative fields, the discrete logarithm problem in groups of points of elliptic curves over finite fields is used. There is a quickly performed operation here - multiplication of an integer 𝑥 to the point 𝐴 (summing the point to itself an integer number of times). In this task, we will ask you to write python function that returns all group elements of a certain elliptic curve over a finite field and performs the points summation. 

1. Write a python function that list all points of elliptic curve $y^2=x^3-12x-6$ over $F_{17}$ (1 points)

In [14]:
from collections import defaultdict
from itertools import product
q = 17
a = (-12) % q
def calc_roots(q):
    roots = defaultdict(list)
    for i in range(q):
        roots[(fast_pow(i, 2)) % q].append(i)
    return roots
# print(calc_roots(q))
roots = calc_roots(q)
def solve(x):
    return ((fast_pow(x, 3) - (12 * x) % q) % q - 6) % q

points = []
for i in range(q):
    points += list(product([i], roots[(solve(i) % q)]))
print(points)

[(1, 0), (3, 6), (3, 11), (5, 5), (5, 12), (6, 6), (6, 11), (7, 7), (7, 10), (8, 6), (8, 11)]


2. Compute the sum of any two points in group above and make sure that result lies within the same group. (0.5 point)

In [57]:
def calc_m(p1, p2, a, q):
    if p1 == p2:
        return (((3 * fast_pow(p1[0], 2) % q) + a) % q ) * inv((2 * p1[1]) % q, q)
    return (p2[1] - p1[1]) * inv((p2[0] - p1[0]), q)

def sum(p1, p2, a, q):
    if (p1[0]==p2[0] and p1[1]!=p2[1]):
        return "inf"
    m = calc_m(p1, p2, a, q)
    m_2 = fast_pow(m, 2)
    return ((m_2 - p1[0] - p2[0]) % q, (m * (2 * p1[0] + p2[0] - m_2) - p1[1]) % q) 


for p1, p2 in product(points, repeat=2):
    if p1 == (1, 0) or p2 == (1, 0):
        continue
    p = sum(p1, p2, a, q)
    if p == "inf":
        continue
    if p not in points:
        print(p1, p2, p)


3. For the point (6,6) define whether it generates the whole group or subgroup of points of this elliptic curve. (2 points)

In [60]:
new_points = [(6, 6)]
generator = (6,6)
new_gen = sum(generator, generator, a, q)
flag = True
while new_gen not in new_points:
    if new_gen == "inf":
        break
    if new_gen not in points:
        print(new_gen, "=> not subgroup or group")
        flag = False
        break
    new_points.append(new_gen)
    new_gen = sum(new_gen, generator, a, q)
    
print(new_points)
if not flag:
    print("not group and not subgroup")
elif points == new_points:
    print("group")
else:
    print("subgroup")


[(6, 6), (1, 0), (6, 11)]
subgroup


4. Compare the number of points with Hasse’s estimate $|N-(p+1)|\leq 2{\sqrt  {p}}$, where $N$ is the number of points on the curve and $p$ is the finite field order. (0.5 point)

### Problem 3 (3 points)

Let us consider the finite field $\mathbb{Z}_p$ with $p=11$. A secret was shared using $(n,3)$-threshold Shamir secret sharing scheme. $4$ shares are known: $(2;9), (3;10), (6;4), (7;10)$. Reconstruct the polynomial $f(x)$ and the common secret $f(0)$.

 Note that each share is given in the form $(x_i; f(x_i))$.